## **CONEXIÓN A DRIVE**

In [ ]:
#para cargar archivos de mi computadora
#from google.colab import files
#uploaded = files.upload()

In [ ]:
#para conectarse directamente al drive
#me quiero conectar al drive porque existe un archivo .txt que contiene el apikey
#dicho apikey se debe actualizar cada cierto tiempo, de acuerdo a lo que la pagina de riot games me indique
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **IMPORTAR LIBRERIAS**

In [ ]:
import requests
import pandas as pd
import json
import time
from datetime import datetime
from timeit import timeit

## **DEFINIR FUNCIONES**

In [ ]:
#API_KEY FUNCTION: va a regresar el api key encriptado en el archivo .txt que esta en drive
def api_key():
    with open('/content/drive/MyDrive/1. IRONHACK/0. Proyecto modulo 2/apiKey.txt') as f:
        contents = f.read()
        api = f'?api_key={contents}'
        return api

In [ ]:
#SUMMONER FUNCTION: va a regresar a todos los summoners de las regiones divididos en sus respectivos tiers
def summoner(api):
    #empty list to append all the tables (aqui va a ser el compilado de toda la info)
    main_df = list()

    #lista con mi top 8 de regiones para los servidores
    #eun1 = europa nordica y este
    #kr1 = Corea del sur
    #na1 = Norteamérica (consideran EUA y Canada)
    #br1 = Brasil
    #la1 = latinoamérica norte (incluido méxico)
    #la2 = latinoamérica sur
    #euw1 = europa oeste
    #jp1 = japon
    region_list = ['eun1','kr','na1','br1','la1','la2','euw1','jp1']

    #defining a list with my 3 top tiers (categorías/ niveles)
    tier_list = ['challengerleagues','grandmasterleagues','masterleagues']

    #loop that will change the url so the 24 combinations are appended to the main_df list
    for region in region_list:
        for tier in tier_list:
            url = f'https://{region}.api.riotgames.com/lol/league/v4/{tier}/by-queue/RANKED_SOLO_5x5{api}'
            response_json = requests.get(url)
            response = response_json.json()
            #print(response)
            response_df = pd.DataFrame(response)
            #adding the column "region" (it will be necessary later)
            response_df['region'] = region
            #print(response_df)
            main_df.append(response_df)
            time.sleep(2)
    summoner = pd.concat(main_df)
    return summoner

In [ ]:
#CHAMPION AND SUMMONER FUNCTION: returns the first version of the summoners df with their champions
def championAndSummoner(api,flat_summoners):
    #empty list to append all the tables
    main_df_1 = list()

    #a for loop to consider all the summoners in the flat_summoners_ordered df
    for i in range(len(flat_summoners)):
        #Con el nuevo codigo, hay un total de 19,359 summoners de 8 regiones y 3 tiers
        if i > 19413:
            break
        #we want the first value of the list given by the .at[i,region]    
        region = flat_summoners.at[i, 'region'][0]
        invocador = flat_summoners.at[i,'summonerId'][0]

        url = f'https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-summoner/{invocador}{api}'
        response_json = requests.get(url)
        response = response_json.json()
        response_df = pd.DataFrame(response)
        main_df_1.append(response_df)
        time.sleep(1)
    champion_and_summoner = pd.concat(main_df_1)
    return champion_and_summoner

In [ ]:
#CHAMPION AND SUMMONER FUNCTION: returns the first version of the summoners df with their champions
def champions():
    champs_json = requests.get('http://ddragon.leagueoflegends.com/cdn/10.23.1/data/en_US/champion.json')
    champs = champs_json.json()
    champs_df = pd.DataFrame(champs)
    return champs_df

In [ ]:
#CHANGE_INDEX FUNCTION: function that will change the index of pd
def change_index(df):
    index = df.shape[0]
    index_final = [*range(0, index, 1)]
    df = df.set_axis([index_final], axis='index')
    return df

In [ ]:
#FLATTEN FUNCTION: will flatten my data (if a column has nested dictionaries)
def flatten(table, column_list):
    for column in column_list:
        flattened = pd.DataFrame(dict(table[column])).transpose()
        columns = [str(col) for col in flattened.columns]
        flattened.columns = [column + '_' + column_name for column_name in columns]
        table = pd.concat([table, flattened], axis=1)
        table = table.drop(column, axis = 1)
    return table

In [ ]:
#MERGE FUNCTION: it will "concat" two tables according an specific value (this case championID)
def merge(df1, df2):
    df = df1.merge(df2,left_on='data_key', right_on='championId')
    return df

In [ ]:
#CSV FUNCTION
#dato: name tiene que terminar con .csv para que se guarde bien
def convert_csv(df,name):
    df.to_csv(name)

## **CLEANING FUNCTIONS**

In [ ]:
#ORDER AND RENAME FUNCTION
def order(df,lista):
    #order columns and delete the ones I won't use
    flat = df[(lista)]
    return flat

In [ ]:
def rename(df,dicc):
    flat_ordered = df.rename(columns=dicc)
    return flat_ordered

## **CÓDIGO - NESTED FUNCTIONS**
SOLO PARA SACAR SUMMONERS

In [ ]:
#mando llamar a mi api_key
api = api_key()

In [ ]:

#tiempo que se tarda con las 8 regiones, y los 3 tiers (tiempo con 24 combinaciones)
start = time.time()

#mando llamar a mi función summoner, va a usar el apikey para conectar con la api de riot games
#esta función me va a crear un df con todos los summoners de mis 8 regiones con su respectivo tier
summoner_df = summoner(api)

#tiempo = 71.5 segundos = 1 min 18 segundos
end = time.time()
print (end - start)

64.7688581943512


In [ ]:
summoner_df

,tier,leagueId,queue,name,entries,region
0,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'l-3F5SK60iy8xRpZhJ1lT7d7EfZKod...,eun1
1,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'OSB7m-MIa4StpbtrV9bIknv6ZA807a...,eun1
2,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'JLSQdP8n1I2BjP-m1SjV9u2rCQ3Aii...,eun1
3,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'VJhQYtSWX-xO60S9OFNJwl30hPog66...,eun1
4,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'sNOhR8ivResBD5KLOopFZCwyjHXkYy...,eun1
...,...,...,...,...,...,...
16,MASTER,caca6dce-d036-3bd3-a7e2-1261d1f9deb1,RANKED_SOLO_5x5,Janna's Defiants,{'summonerId': 'gq5_1zY1SupXZEp1AYYOtKqD3Dekyz...,jp1
17,MASTER,caca6dce-d036-3bd3-a7e2-1261d1f9deb1,RANKED_SOLO_5x5,Janna's Defiants,{'summonerId': 'FoYRUtI9MaRBup3iC7hvxuebMnnLna...,jp1
18,MASTER,caca6dce-d036-3bd3-a7e2-1261d1f9deb1,RANKED_SOLO_5x5,Janna's Defiants,{'summonerId': 'LB033dyeQEbgQ5Ch-b6uDg1KlaaLDX...,jp1
19,MASTER,caca6dce-d036-3bd3-a7e2-1261d1f9deb1,RANKED_SOLO_5x5,Janna's Defiants,{'summonerId': 'hoMVmajIV0VxoF7ak9oe2iM7jjIYXe...,jp1


In [ ]:
#mando llamar la función change_index para cambiar los index de mi df
summoner_df = change_index(summoner_df)
summoner_df

,tier,leagueId,queue,name,entries,region
0,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'l-3F5SK60iy8xRpZhJ1lT7d7EfZKod...,eun1
1,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'OSB7m-MIa4StpbtrV9bIknv6ZA807a...,eun1
2,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'JLSQdP8n1I2BjP-m1SjV9u2rCQ3Aii...,eun1
3,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'VJhQYtSWX-xO60S9OFNJwl30hPog66...,eun1
4,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,{'summonerId': 'sNOhR8ivResBD5KLOopFZCwyjHXkYy...,eun1
...,...,...,...,...,...,...
19408,MASTER,caca6dce-d036-3bd3-a7e2-1261d1f9deb1,RANKED_SOLO_5x5,Janna's Defiants,{'summonerId': 'gq5_1zY1SupXZEp1AYYOtKqD3Dekyz...,jp1
19409,MASTER,caca6dce-d036-3bd3-a7e2-1261d1f9deb1,RANKED_SOLO_5x5,Janna's Defiants,{'summonerId': 'FoYRUtI9MaRBup3iC7hvxuebMnnLna...,jp1
19410,MASTER,caca6dce-d036-3bd3-a7e2-1261d1f9deb1,RANKED_SOLO_5x5,Janna's Defiants,{'summonerId': 'LB033dyeQEbgQ5Ch-b6uDg1KlaaLDX...,jp1
19411,MASTER,caca6dce-d036-3bd3-a7e2-1261d1f9deb1,RANKED_SOLO_5x5,Janna's Defiants,{'summonerId': 'hoMVmajIV0VxoF7ak9oe2iM7jjIYXe...,jp1


In [ ]:
#la columna entries tiene un "nested dictionary", necesito abrirlo para obtener el summonerId
#defino la columna que necesito "abrir"
nested_column = ['entries']

#mando llamar mi función dandole como argumentos el df y la columna
flat_summoners = flatten(summoner_df, nested_column)
flat_summoners.head()

,tier,leagueId,queue,name,region,entries_summonerId,entries_summonerName,entries_leaguePoints,entries_rank,entries_wins,entries_losses,entries_veteran,entries_inactive,entries_freshBlood,entries_hotStreak
0,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,eun1,l-3F5SK60iy8xRpZhJ1lT7d7EfZKodz-6iU1DRkbIwSPl3U,R4 Coatpustmain,791,I,269,232,True,False,False,False
1,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,eun1,OSB7m-MIa4StpbtrV9bIknv6ZA807aQDAT45OmpO9iJGsxc,Kizuro,728,I,153,108,False,False,False,True
2,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,eun1,JLSQdP8n1I2BjP-m1SjV9u2rCQ3Aii9OSKAif0v9RmEQ5_c,Mezram,553,I,228,133,False,False,False,False
3,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,eun1,VJhQYtSWX-xO60S9OFNJwl30hPog66s2cSLvnNNi3xbJ6ng,F1ko,532,I,135,101,False,False,False,False
4,CHALLENGER,0ef324f6-1ba0-3357-829e-4793b7be8601,RANKED_SOLO_5x5,Renekton's Maulers,eun1,sNOhR8ivResBD5KLOopFZCwyjHXkYycLOdiqKxewSC9MaGU,Anshej,622,I,481,443,False,False,False,True


In [ ]:
#creo una lista de las columnas que quiero que aparezcan en mi df final, para acomodarlas en ese orden
lista = ['entries_summonerId','entries_summonerName','region','tier',
                'entries_wins','entries_losses','leagueId','name','entries_leaguePoints']

flat_summoners = order(flat_summoners, lista)
flat_summoners.head()

,entries_summonerId,entries_summonerName,region,tier,entries_wins,entries_losses,leagueId,name,entries_leaguePoints
0,l-3F5SK60iy8xRpZhJ1lT7d7EfZKodz-6iU1DRkbIwSPl3U,R4 Coatpustmain,eun1,CHALLENGER,269,232,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,791
1,OSB7m-MIa4StpbtrV9bIknv6ZA807aQDAT45OmpO9iJGsxc,Kizuro,eun1,CHALLENGER,153,108,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,728
2,JLSQdP8n1I2BjP-m1SjV9u2rCQ3Aii9OSKAif0v9RmEQ5_c,Mezram,eun1,CHALLENGER,228,133,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,553
3,VJhQYtSWX-xO60S9OFNJwl30hPog66s2cSLvnNNi3xbJ6ng,F1ko,eun1,CHALLENGER,135,101,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,532
4,sNOhR8ivResBD5KLOopFZCwyjHXkYycLOdiqKxewSC9MaGU,Anshej,eun1,CHALLENGER,481,443,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,622


In [ ]:
#creo un diccionario para cambiar el nombre de mis columnas con la función rename
this_dict ={'entries_summonerId':'summonerId','entries_summonerName':'summonerName', 
            'entries_leaguePoints':'leaguePoints',
            'entries_wins':'wins','entries_losses':'losses','name':'leagueName'}

flat_summoners = rename(flat_summoners, this_dict)
flat_summoners.head()

,summonerId,summonerName,region,tier,wins,losses,leagueId,leagueName,leaguePoints
0,l-3F5SK60iy8xRpZhJ1lT7d7EfZKodz-6iU1DRkbIwSPl3U,R4 Coatpustmain,eun1,CHALLENGER,269,232,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,791
1,OSB7m-MIa4StpbtrV9bIknv6ZA807aQDAT45OmpO9iJGsxc,Kizuro,eun1,CHALLENGER,153,108,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,728
2,JLSQdP8n1I2BjP-m1SjV9u2rCQ3Aii9OSKAif0v9RmEQ5_c,Mezram,eun1,CHALLENGER,228,133,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,553
3,VJhQYtSWX-xO60S9OFNJwl30hPog66s2cSLvnNNi3xbJ6ng,F1ko,eun1,CHALLENGER,135,101,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,532
4,sNOhR8ivResBD5KLOopFZCwyjHXkYycLOdiqKxewSC9MaGU,Anshej,eun1,CHALLENGER,481,443,0ef324f6-1ba0-3357-829e-4793b7be8601,Renekton's Maulers,622


In [ ]:
#comprobación 1: todos deben ser objects
flat_summoners.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 19413 entries, (0,) to (19412,)
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   summonerId    19413 non-null  object
 1   summonerName  19413 non-null  object
 2   region        19413 non-null  object
 3   tier          19413 non-null  object
 4   wins          19413 non-null  object
 5   losses        19413 non-null  object
 6   leagueId      19413 non-null  object
 7   leagueName    19413 non-null  object
 8   leaguePoints  19413 non-null  object
dtypes: object(9)
memory usage: 2.8+ MB


In [ ]:
#comprobación 2: deben estar las 8 regiones
flat_summoners['region'].unique()

array(['eun1', 'kr', 'na1', 'br1', 'la1', 'la2', 'euw1', 'jp1'],
      dtype=object)

In [ ]:
#comprobación 3: deben estar los 3 tier
flat_summoners['tier'].unique()

array(['CHALLENGER', 'GRANDMASTER', 'MASTER'], dtype=object)

In [ ]:
#vamos a guardar el primer df, es importante que el nombre termine con .csv
name_csv = 'summoners_8_regions.csv'

summoners_df = convert_csv(flat_summoners,name_csv)

## Summoners con sus respectivos champions

In [ ]:
#ESTA FUNCION SE TARDA MUCHO!!!! PERO ES NECESARIA
#tiempo que se tarda con las 8 regiones, y los 3 tiers y los 152 champions (tiempo con 3648 combinaciones)
start = time.time()

#mando llamar a mi función championAndSummoner, va a usar el apikey para conectar con la api de riot games y mi df anterior
#esta función me va a crear un df con todos los summoners de mis 8 regiones, 3 tiers y sus posibles elecciones de los 152 champions
champions_summoner = championAndSummoner(api,flat_summoners)

#tiempo = 33964 segundos = 9 horas 43 minutos
end = time.time()
print (end - start)

In [ ]:
champions_summoner

In [ ]:
#mando llamar la función change_index para cambiar los index de mi df
champions_summoner = change_index(champions_summoner)
champions_summoner

In [ ]:
#corroboro que championId sea object
champions_summoner.info()

In [ ]:
#I need to change the values of the column "championId" from float64 to str
champions_summoner['championId'] = champions_summoner['championId'].astype('str')

In [ ]:
#compruebo
champions_summoner.info()

In [ ]:
#vamos a guardar el segundo df
name_csv = 'champs_summoners.csv'

champions_summoner_df = convert_csv(champions_summoner_df,name_csv)

In [ ]:
#a este codigo le falta la parte de los champions
#ese codigo esta en el archivo de crear csv Lol